# DATA SCIENCE CHALLENGE #4 - SERIES DE TIEMPO

+ Pronosticar las ventas de un producto o de una categoría es un desafío recurrente para cualquier ecommerce;
+ En este caso, el desafío va a ser pronosticar la cantidad de unidades diarias que van a vender 3 categorías distintas de MELI;
+ El dataset `series.csv` tiene las ventas en unidades diarias de 3 categorías que poseen un id único y su fecha de venta;
+ El objetivo de este desafío es construir un modelo de forecast que permita estimar las ventas de 3 semanas a nivel diario utilizando la historia de ventas de la categoría. Es decir, predecir las ventas de los siguientes 21 días. Las métricas y la medición de la performance del forecast son un punto clave de este desafío.

# Loading modules and functions

In [2]:
import pandas as pd

## Loading the data

In [3]:
df = pd.read_csv(filepath_or_buffer = '../data/series.csv')